In [20]:
import cv2
from PIL import Image
import numpy as np
from keras_facenet import FaceNet


# Load the pre-trained Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')



In [21]:

model = FaceNet()


In [27]:
def get_faces(image_path):
    # Convert the frame to grayscale (needed for face detection)
    image=cv2.imread(image_path)
    
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Find the largest face based on area (width * height)
    largest_face = max(faces, key=lambda rect: rect[2] * rect[3])  # rect[2] = width, rect[3] = height

    # Extract coordinates of the largest face
    (x, y, w, h) = largest_face
    face_image = image[y:y+h, x:x+w]
    return face_image

In [28]:
def get_face_embedding(image):
    face_image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    face_image_pil = Image.fromarray(face_image_rgb)  # Convert to PIL Image
    face_image_pil = face_image_pil.resize((160, 160))  # Resize to 160x160
    face_image_array = np.array(face_image_pil)  # Convert back to numpy array
    embeddings = model.embeddings([face_image_array])  # Get face embeddings
    return embeddings

In [24]:
known_face_embeddings = {
    'Johnny Deep': get_face_embedding(get_faces('C:/Users/barra/Downloads/johny_2.jfif')),
    'Jason Statham':get_face_embedding(get_faces('C:/Users/barra/Downloads/Jason Statham.jfif')),
    'Ronaldo':get_face_embedding(get_faces('C:/Users/barra/Downloads/ronaldo.jfif')),

}

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


In [16]:

from sklearn.metrics.pairwise import cosine_similarity

# Function to compare face embeddings (cosine similarity)
def get_similarity(embedding1, embedding2):
    embedding1 = np.reshape(embedding1, (1, -1))  
    embedding2 = np.reshape(embedding2, (1, -1)) 
    similarity = cosine_similarity(embedding1, embedding2)[0][0]  # Get the similarity value
    return similarity 

In [25]:
em1=known_face_embeddings['Jason Statham']
em2=known_face_embeddings['Ronaldo']
get_similarity(em1,em2)

-0.031282507

In [30]:

# Start video capture (use 0 for the default webcam)
video_capture = cv2.VideoCapture(0) 

if not video_capture.isOpened():
    print("Error: Could not open video stream or file")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    if not ret:
        print("Error: Failed to capture image")
        break

    # Convert the frame to grayscale (needed for face detection)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        face_image = gray_frame[y:y+h, x:x+w]
        embedded_face=get_face_embedding(face_image)
        threshold=0.5        
        for name, known_embedding_face in known_face_embeddings.items():
            similarity=get_similarity(embedded_face,known_embedding_face)
            if  similarity> threshold:
                print(name)
                print(similarity)
                name = name
                break
            else:
                 name = "Unknown"
                 print(similarity)



        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    # Display the frame with the detected faces
    cv2.imshow('Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the window
video_capture.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
-0.043927193
-0.15147811
0.060143836
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
-0.03737777
-0.1329395
-0.081832886
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
0.04578995
-0.078528166
0.033465266
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
0.061059043
-0.18724854
0.07407988
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
0.018559419
-0.10070203
0.0018997863
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
-0.04308879
-0.08236198
-0.058700487
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
-0.028114455
-0.030895155
-0.04977937
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
-0.030976374
-0.17934945
0.013079971
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
-0.052538216
-0.1879184
-0.015897442
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
-0.04824
-0.20990571
0.017690014
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
-0.17031848
-0.13838556
0.030748688
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
-0.1511878
-0.17015976
-0.004409856
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
-0.18889627
-0.32288063
0.10868773
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/ste